In [121]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

In [122]:
dataset = 'model/keypoint_classifier/output.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.keras'
model_save_path1 = 'model/keypoint_classifier/keypoint_classifier.hdf5'

In [133]:
NUM_CLASSES = 27

In [134]:
# X_dataset = np.loadtxt('model/keypoint_classifier/output.csv', delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [135]:
# y_dataset = np.loadtxt('model/keypoint_classifier/keypoint.csv', delimiter=',', usecols=(0))

In [136]:
# print(len(y_dataset))

In [137]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset using pandas
df = pd.read_csv(dataset)

# Inspect the first few rows of the dataset
print(df.shape)


(57083, 43)


In [153]:
# Assuming the label is in the first column
X = df.iloc[:, 1:].values  # Features
y = df.iloc[:, 0].values   # Labels
print(y)

['A' 'A' 'A' ... 'Z' 'Z' 'Z']


In [154]:
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_encoded = label_encoder.fit_transform(y)
y = to_categorical(y_encoded)
print(y_encoded.shape)

(57083,)


In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=RANDOM_SEED)

# モデル構築

In [156]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Input((21 * 2, )),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(20, activation='relu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
# ])

# y_he = to_categorical(y_train)
# y_test_he = to_categorical(y_test)

num_classes =len(np.unique(y_test))
print(num_classes)

model = Sequential([
    Dense(128, input_shape=(42,), activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')  # Use the variable num_classes
])


2


C:\Users\Marianna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [158]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_83 (Dense)                │ (None, 128)            │         5,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_63          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_63 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_64          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_64 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_65          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_65 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_86 (Dense)                │ (None, 27)             │           891 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,627 (68.86 KB)

 Trainable params: 17,179 (67.11 KB)

 Non-trainable params: 448 (1.75 KB)

In [159]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [160]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [161]:
model.fit(
    X_train,
    y_train,    # y_train should be one-hot encoded
    epochs=500,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/500
329/335 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2517 - loss: 2.6948
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
335/335 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.2551 - loss: 2.6805 - val_accuracy: 0.8107 - val_loss: 1.0742
Epoch 2/500
333/335 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6640 - loss: 1.0632
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
335/335 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6644 - loss: 1.0620 - val_accuracy: 0.9639 - val_loss: 0.2599
Epoch 3/500
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7910 - loss: 0.6737
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
335/335 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7911 - loss: 0.6735 - val_accuracy: 0.9690 - val_loss: 0.1566
Epoch 4/500
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8466 - loss: 0.4954
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier

In [163]:
# モデル評価
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9901 - loss: 0.0453


In [164]:
# 保存したモデルのロード
model = tf.keras.models.load_model(model_save_path)

In [165]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
[5.58250113e-09 1.61958869e-12 7.96788913e-10 1.60108204e-10
 1.44684189e-12 1.26179856e-07 1.01129425e-08 9.50837830e-07
 1.57781912e-08 1.23941319e-08 9.99889970e-01 1.14058656e-08
 5.75851367e-08 9.31265820e-09 2.40508613e-10 2.19430563e-09
 1.39270193e-12 3.48084728e-10 1.44354098e-10 4.94705743e-09
 6.27356421e-05 4.48914107e-05 8.06216036e-08 2.38129988e-10
 2.50060164e-11 1.02505021e-06 1.12515545e-10]
10


In [166]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, classification_report

# def print_confusion_matrix(y_true, y_pred, report=True):
#     labels = sorted(list(set(y_true)))
#     cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
#     df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
#     fig, ax = plt.subplots(figsize=(7, 6))
#     sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
#     ax.set_ylim(len(set(y_true)), 0)
#     plt.show()
    
#     if report:
#         print('Classification Report')
#         print(classification_report(y_test, y_pred))

# Y_pred = model.predict(X_test)
# y_pred = np.argmax(Y_pred, axis=1)

# print_confusion_matrix(y_test, y_pred)

# Tensorflow-Lite用のモデルへ変換

In [167]:

model.save(model_save_path, include_optimizer=False)

In [21]:
# モデルを変換(量子化)
# tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quantized_model = converter.convert()

# open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\Marianna\AppData\Local\Temp\tmp559txvn5\assets


INFO:tensorflow:Assets written to: C:\Users\Marianna\AppData\Local\Temp\tmp559txvn5\assets


: 

# 推論テスト

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

NameError: name 'tf' is not defined

In [ ]:
# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# 推論実施
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

Wall time: 0 ns


In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[0.7729778  0.16973573 0.05728643]
0
